In [1]:
import pandas
from sklearn.model_selection import train_test_split
import math
import numpy as np

4 neurons in input layer,
3 neurons in output layer,
2 nodes in 1 hidden layer

In [2]:
iris = pandas.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',names=['sepal.length','sepal.width','petal.length','petal.width','species'])
X = iris.loc[:, iris.columns != 'species']
X =(X-X.min())/(X.max()-X.min())
y = pandas.get_dummies(iris['species'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)


### Forward Propogation

$$f(t) = \frac{1}{1 + e^{-t}}$$
$\hat{y} = f(f(XW^{(1)} +  B^{(1)})W^{(2)}+B^{(2)})$

In [3]:
batch_size =  120
input_layer_size = 4
output_layer_size = 3
hidden_layer_size = 2

def sigmoid(z):
    return(1/(1 + np.exp(z)))
           
def sigmoid_prime(z):
    return(np.exp(-z)/(1 + np.exp(z))**2)

def cost(y,y_pred):
    return(0.5 * np.sum(np.square(np.matrix(y) - np.matrix(y_pred))))

def initialize_weights(rows,columns):
    W = []
    for row in range(rows):
        W.append(np.random.uniform(low = -.5, high = .5, size = columns))
    return(np.matrix(W))

W1 = initialize_weights(input_layer_size,hidden_layer_size)
W2 = initialize_weights(hidden_layer_size,output_layer_size)
b1 = np.random.uniform(size = hidden_layer_size) 
B1 = np.matrix([b1 for i in range(batch_size)])
b2 = np.random.uniform(size = output_layer_size) 
B2 = np.matrix([b2 for i in range(batch_size)])

Z2 = X_train.dot(W1)
A2 = sigmoid(Z2 + B1)
Z3 = A2.dot(W2)
y_pred = sigmoid(Z3 + B2)
cost(y_train, y_pred)

39.98242633619949

### Back Propogation

Cost Function:
$$J = 0.5 \sum (y - \hat{y})^2 = 0.5 \sum (y - f(f(XW^{(1)} +  B^{(1)})W^{(2)}+B^{(2)}))^2$$

Gradient of Cost Function:
$$\frac{\partial J }{\partial W^{(1)}} = X^{T}-(y - \hat{y})^2  *f'(z^{(3)}+B^{(1)})*W^{(2)T}f'(z^{(3)})$$
$$\frac{\partial J }{\partial W^{(2)}} = a^{(2)}-(y - \hat{y})^2  *f'(z^{(3)}+B^{(1)})$$
$$\frac{\partial J }{\partial B^{(1)}} = a^{(2)}-(y - \hat{y})^2  *f'(z^{(3)}+B^{(1)})$$
$$\frac{\partial J }{\partial B^{(2)}} = a^{(2)}-(y - \hat{y})^2  *f'(z^{(3)}+B^{(1)})$$